In [1]:
import ufal.udpipe
from ufal.udpipe import Trainer, Evaluator
import os
import conllu

In [2]:
#Implementing a processing pipeline, which inclides tokenization, morphological tagging and syntactic parsing
#Source: https://github.com/ufal/udpipe/tree/master/bindings/python/examples

class Model:
    def __init__(self, model):
        """Load given model."""
        self.model = model
        if not self.model:
            raise Exception("Cannot load UDPipe model from file '%s'" % model)

    def tokenize(self, text):
        """Tokenize the text and return list of ufal.udpipe.Sentence-s."""
        tokenizer = self.model.newTokenizer(self.model.DEFAULT)
        if not tokenizer:
            raise Exception("The model does not have a tokenizer")
        return self._read(text, tokenizer)

    def read(self, text, format):
        """Load text in the given format (conllu|horizontal|vertical) and return list of ufal.udpipe.Sentence-s."""
        input_format = ufal.udpipe.InputFormat.newInputFormat(format)
        if not input_format:
            raise Exception("Cannot create input format '%s'" % format)
        return self._read(text, input_format)

    def _read(self, text, input_format):
        input_format.setText(text)
        error = ufal.udpipe.ProcessingError()
        sentences = []

        sentence = ufal.udpipe.Sentence()
        while input_format.nextSentence(sentence, error):
            sentences.append(sentence)
            sentence = ufal.udpipe.Sentence()
        if error.occurred():
            raise Exception(error.message)

        return sentences

    def tag(self, sentence):
        """Tag the given ufal.udpipe.Sentence (inplace)."""
        self.model.tag(sentence, self.model.DEFAULT)

    def parse(self, sentence):
        """Parse the given ufal.udpipe.Sentence (inplace)."""
        self.model.parse(sentence, self.model.DEFAULT)

    def write(self, sentences, format):
        """Write given ufal.udpipe.Sentence-s in the required format (conllu|horizontal|vertical)."""

        output_format = ufal.udpipe.OutputFormat.newOutputFormat(format)
        output = ''
        for sentence in sentences:
            output += output_format.writeSentence(sentence)
        output += output_format.finishDocument()
        return output
    
def process(raw_model, text):
    model = Model(raw_model)
    sentences = model.tokenize(text)
    for sent in sentences:
        model.tag(sent)
        model.parse(sent)
    output = model.write(sentences, "conllu")
    return output

### Обучение моделей и определение их метрик проводилось с помощью скомпилированной версии udpipe. Код:

udpipe --train projective.udpipe UD_Russian-SynTagRus/ru_syntagrus-ud-train-short.conllu --tokenizer epochs=10 --tagger iterations=10 --parser iterations=10;transition_system=projective

udpipe --train swap.output UD_Russian-SynTagRus/ru_syntagrus-ud-train-short.conllu --tokenizer epochs=10 --tagger iterations=10 --parser iterations=10;transition_system=swap

udpipe --train link2.output UD_Russian-SynTagRus/ru_syntagrus-ud-train-short.conllu --tokenizer epochs=10 --tagger iterations=10 --parser iterations=10;transition_system=link2

udpipe --accuracy --parse projective.output UD_Russian-SynTagRus/ru_syntagrus-ud-dev.conllu

udpipe --accuracy --parse swap.output UD_Russian-SynTagRus/ru_syntagrus-ud-dev.conllu

udpipe --accuracy --parse link2.output UD_Russian-SynTagRus/ru_syntagrus-ud-dev.conllu

In [6]:
#Parsing the test text using the models trained on 2018 data

model_projective = ufal.udpipe.Model.load('projective.output')
model_swap = ufal.udpipe.Model.load('swap.output')
model_link2 = ufal.udpipe.Model.load('link2.output')

In [5]:
with open('UD_Russian-SynTagRus/ru_syntagrus-ud-dev.txt', 'r', encoding = 'utf-8') as f:
    input_text = f.read()[:10000]
    parsed_projective = process(model_projective, input_text)
    print('done with projective parsing')
    parsed_swap = process(model_swap, input_text)
    print('done with swap parsing')
    parsed_link2 = process(model_link2, input_text)
    print('done with link2 parsing')

done with projective parsing
done with swap parsing
done with link2 parsing


In [ ]:
proj_conll = conllu.parse(parsed_projective)
swap_conll = conllu.parse(parsed_swap)
link2_conll = conllu.parse(parsed_link2)

In [40]:
disagr = []
sent = 0
for sentproj, sentswap, sentlink2 in zip(proj_conll, swap_conll, link2_conll):
    sent += 1
    for wordproj, wordswap, wordlink2 in zip(sentproj, sentswap, sentlink2):
        if (wordproj['deprel'] != wordswap['deprel']) or (wordproj['deprel'] != wordlink2['deprel']) or (wordswap['deprel'] != wordlink2['deprel']):
            disagr.append((sentproj, sentswap, sentlink2))
            break

In [42]:
for el in disagr:
    print(el[0].serialize())
    for sent in el:
        sent.to_tree().print_tree()

# sent_id = 1
# text = Алгоритм, от имени учёного аль-Хорезми, - точный набор инструкций, описывающих порядок действий исполнителя для достижения результата решения задачи за конечное время.
1	Алгоритм	алгоритм	NOUN	_	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing	10	nsubj	_	SpaceAfter=No
2	,	,	PUNCT	_	_	1	punct	_	_
3	от	от	ADP	_	_	4	case	_	_
4	имени	имя	NOUN	_	Animacy=Inan|Case=Dat|Gender=Neut|Number=Sing	1	nmod	_	_
5	учёного	учёный	ADJ	_	Case=Gen|Degree=Pos|Gender=Neut|Number=Sing	6	amod	_	_
6	аль-Хорезми	аль-Хорез	NOUN	_	Animacy=Inan|Case=Ins|Gender=Fem|Number=Plur	4	nmod	_	SpaceAfter=No
7	,	,	PUNCT	_	_	6	punct	_	_
8	-	-	PUNCT	_	_	1	punct	_	_
9	точный	точный	ADJ	_	Case=Nom|Degree=Pos|Gender=Masc|Number=Sing	10	amod	_	_
10	набор	набор	NOUN	_	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing	0	root	_	_
11	инструкций	инструкция	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Plur	10	nmod	_	SpaceAfter=No
12	,	,	PUNCT	_	_	13	punct	_	_
13	описывающих	описывать	VERB	_	Aspect=Imp|Case=Gen|Number=

        (deprel:nmod) form:работы lemma:работа upostag:NOUN [7]
    (deprel:advmod) form:однозначно lemma:однозначно upostag:ADV [8]
    (deprel:obl) form:состоянием lemma:состояние upostag:NOUN [10]
        (deprel:nmod) form:системы lemma:система upostag:NOUN [11]
    (deprel:punct) form:. lemma:. upostag:PUNCT [12]
(deprel:root) form:определяется lemma:определяваться upostag:VERB [9]
    (deprel:obl) form:момент lemma:момент upostag:NOUN [3]
        (deprel:case) form:В lemma:в upostag:ADP [1]
        (deprel:det) form:каждый lemma:каждый upostag:DET [2]
    (deprel:obl) form:времени lemma:время upostag:NOUN [4]
    (deprel:nsubj) form:шаг lemma:шаг upostag:NOUN [6]
        (deprel:amod) form:следующий lemma:следующий upostag:ADJ [5]
        (deprel:nmod) form:работы lemma:работа upostag:NOUN [7]
    (deprel:advmod) form:однозначно lemma:однозначно upostag:ADV [8]
    (deprel:obl) form:состоянием lemma:состояние upostag:NOUN [10]
        (deprel:nmod) form:системы lemma:система upos

            (deprel:punct) form:) lemma:) upostag:PUNCT [9]
        (deprel:conj) form:arithmos lemma:arithmos upostag:PROPN [11]
            (deprel:cc) form:и lemma:и upostag:CCONJ [10]
        (deprel:parataxis) form:число lemma:чисти upostag:VERB [13]
            (deprel:punct) form:( lemma:( upostag:PUNCT [12]
            (deprel:punct) form:) lemma:) upostag:PUNCT [14]
            (deprel:punct) form:. lemma:. upostag:PUNCT [15]
# sent_id = 58
# text = Из такого объяснения не очень ясно, почему числа именно "больные".
1	Из	из	ADP	_	_	3	case	_	_
2	такого	такой	DET	_	Case=Gen|Gender=Neut|Number=Sing	3	det	_	_
3	объяснения	объяснение	NOUN	_	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing	6	obl	_	_
4	не	не	PART	_	_	5	advmod	_	_
5	очень	очень	ADV	_	Degree=Pos	6	advmod	_	_
6	ясно	ясно	ADV	_	Degree=Pos	9	advmod	_	SpaceAfter=No
7	,	,	PUNCT	_	_	9	punct	_	_
8	почему	почему	ADV	_	Degree=Pos	9	advmod	_	_
9	числа	чисти	VERB	_	Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|V

        (deprel:case) form:во lemma:в upostag:ADP [7]
        (deprel:amod) form:многие lemma:много upostag:NUM [8]
        (deprel:amod) form:европейские lemma:европейский upostag:ADJ [9]
    (deprel:punct) form:. lemma:. upostag:PUNCT [11]
(deprel:root) form:вошло lemma:войти upostag:VERB [6]
    (deprel:advmod) form:Именно lemma:именно upostag:ADV [1]
    (deprel:obl) form:значении lemma:значение upostag:NOUN [4]
        (deprel:case) form:в lemma:в upostag:ADP [2]
        (deprel:det) form:таком lemma:такой upostag:DET [3]
    (deprel:nsubj) form:оно lemma:оно upostag:PRON [5]
    (deprel:obl) form:языки lemma:языка upostag:NOUN [10]
        (deprel:case) form:во lemma:в upostag:ADP [7]
        (deprel:amod) form:многие lemma:много upostag:NUM [8]
        (deprel:amod) form:европейские lemma:европейский upostag:ADJ [9]
    (deprel:punct) form:. lemma:. upostag:PUNCT [11]
# sent_id = 85
# text = Например, с пометкой "устар." оно присутствует в представительном словаре английского яз

In [8]:
with open('nonproj.txt', 'r', encoding = 'utf-8') as f:
    nonproj = f.read()
    nonproj_projective = conllu.parse(process(model_projective, nonproj))
    print('done with projective parsing')
    nonproj_swap = conllu.parse(process(model_swap, nonproj))
    print('done with swap parsing')
    nonproj_link2 = conllu.parse(process(model_link2, nonproj))
    print('done with link2 parsing')

done with projective parsing
done with swap parsing
done with link2 parsing


In [10]:
for sentproj, sentswap, sentlink2 in zip(nonproj_projective, nonproj_swap, nonproj_link2):
    sentproj.to_tree().print_tree()
    sentswap.to_tree().print_tree()
    sentlink2.to_tree().print_tree()
    print('next one')

(deprel:root) form:любит lemma:любить upostag:VERB [3]
    (deprel:advmod) form:Пиво lemma:пиво upostag:ADV [1]
    (deprel:nsubj) form:он lemma:он upostag:PRON [2]
    (deprel:obj) form:темное lemma:темный upostag:ADJ [4]
    (deprel:punct) form:. lemma:. upostag:PUNCT [5]
(deprel:root) form:любит lemma:любить upostag:VERB [3]
    (deprel:advmod) form:Пиво lemma:пиво upostag:ADV [1]
    (deprel:nsubj) form:он lemma:он upostag:PRON [2]
    (deprel:obl) form:темное lemma:темный upostag:ADJ [4]
    (deprel:punct) form:. lemma:. upostag:PUNCT [5]
(deprel:root) form:любит lemma:любить upostag:VERB [3]
    (deprel:advmod) form:Пиво lemma:пиво upostag:ADV [1]
    (deprel:nsubj) form:он lemma:он upostag:PRON [2]
    (deprel:obl) form:темное lemma:темный upostag:ADJ [4]
    (deprel:punct) form:. lemma:. upostag:PUNCT [5]
next one
(deprel:root) form:нужно lemma:нужно upostag:ADV [3]
    (deprel:obj) form:Учителя lemma:Учитель upostag:NOUN [1]
    (deprel:advmod) form:очень lemma:очень upostag:A